In [ ]:
import math

import pandas as pd

data = pd.read_parquet("../data/cache/raw_data_db_cache.parquet")
data.shape

In [ ]:
from omegaconf import OmegaConf
from src.data_pipeline import prepare_time_series_segments

crop_start_s = 5
crop_end_s = 5
resample_rate_hz = 50
segment_size_s = 5
overlap_s = 0

cfg = {
    "preprocessing": {
        "crop": {
            "start_seconds": crop_start_s,
            "end_seconds": crop_end_s,
        },
        "resample_rate_hz": resample_rate_hz,
        "segment_size_seconds": segment_size_s,
        "overlap_seconds": overlap_s,
    }
}

cfg = OmegaConf.create(cfg)

segments_df = prepare_time_series_segments(data,
                                           cfg)

segments_df.shape

In [ ]:
def get_session_lengths(df):
    return (df.groupby("session_id")
            .agg({"session_id": "count"})
            .rename(columns={"session_id": "count"}) / resample_rate_hz)


get_session_lengths(segments_df).plot.hist()
get_session_lengths(segments_df).describe()
# get_session_lengths(segments_df).to_dict()["count"]

In [ ]:
file_length_limit_s = 5 * 60  # 5 minutes
max_count_segments = math.floor(file_length_limit_s / segment_size_s)
max_count_segments

In [ ]:
segments_truncated_df = segments_df.groupby('session_id').apply(
    lambda x: x[x['segment_id'].isin(
        x['segment_id'].drop_duplicates().sample(n=min(len(x['segment_id'].drop_duplicates()), max_count_segments))
    )]
).reset_index(drop=True)

(segments_truncated_df.groupby("session_id")
 .agg({"session_id": "count"})
 .rename(columns={"session_id": "count"})).describe()

In [ ]:
get_session_lengths(segments_df).plot.hist()
get_session_lengths(segments_df).describe()

In [ ]:
get_session_lengths(segments_truncated_df).plot.hist()
get_session_lengths(segments_truncated_df).describe() 